Importando bibliotecas

In [1]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument 

import pandas as pd
import numpy as np
import collections
import logging
import random

Configurando nível de logs

In [2]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Lendo arquivos de texto

In [4]:
rfzd_claims = pd.read_parquet('../dataset/refined/rfzd_telecom_pandemic_claims.parquet.snappy')

Dividindo entre conjunto de treino e teste

In [5]:
train_set = rfzd_claims.sample(frac=0.8, random_state=51)
test_set = rfzd_claims.drop(train_set.index)

train_set.reset_index(inplace=True, drop=True)
test_set.reset_index(inplace=True, drop=True)

In [14]:
train_set

,tokens_customer_claim,id
0,"[ha, meses, vinha, sendo, cobrado, por, um, pr...",bc0916bc66ab0637ae1a2cd905c552d222062d1f
1,"[boa, noite, estou, com, dificuldades, em, man...",877ec71696b8b7c6c1691e1a71f3b252fca69def
2,"[possuo, telefone, fixo, contratado, com, oi, ...",e4ccb9548d28f890db220def3d285ba27ee778bc
3,"[tenho, alguns, planos, que, quero, fazer, alt...",e67dde9bc5862af426fd48c0dfab5865b6a76c70
4,"[no, mes, deste, ano, entrei, em, contato, com...",a5b4e5f60aa7cc7201a1154ba87d9916df260657
...,...,...
22630,"[apos, diversos, problemas, com, linha, refere...",c46b013b909442753f97be6a3dd0fb4a04e3c941
22631,"[eu, fui, ate, na, claro, pra, fazer, portabil...",21514f671d0c8ec1b7b69cfd95a08b0f4dbb17d5
22632,"[em, depois, de, alguns, problemas, tecnicos, ...",69e5d8f39e118af86eaa4109967ac591e50c7136
22633,"[tim, nao, me, resaciu, valor, que, paguei, me...",cbfcb74c4bf0a873006a300b890786d018ef3d70


Criando tag para armazenar marcar cada documento.

In [6]:
train_documents = []

for index, data in train_set.iterrows():
    train_documents.append(TaggedDocument(data['tokens_customer_claim'], [index]))

In [7]:
test_documents = test_set['tokens_customer_claim'].values.tolist()

Definindo modelo Doc2vec

In [8]:
model = Doc2Vec(
    vector_size=400, # tamanho do vetor de embedding
    epochs=8, # número de iterações no dataset inteiro para o treinamento
    min_count=5, # quantidade mínima de repetições palavras para entrarem no treinamento 
    sample=10**-5, # frequência teórica para ponderar palavras muito frequêntes
    hs=0, # utilizando amostragem negativa para acelarar o treinamento
    negative=20, # determina que 20 palavras aleatórias serão utilizadas para treinar a palavra predita
    dm=1, # usando modelo PV-DM
    dm_concat=1, # concatenando vetor de parágrafo com de palavras
    dbow_words=1, # treinando também vetor de palavras word2vec
    shrink_windows=True, # define a quantidade de palavras dinamicamente durante o treinamento
    workers=8, # numero de cores utilizados no paralelismo
    seed=51, # fixando a aleatoriada para deixar o modelo reprodutível
)

2024-09-19 17:46:28,005 : INFO : using concatenative 4400-dimensional layer1
2024-09-19 17:46:28,008 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/c,d400,n20,w5,mc5,s1e-05,t8>', 'datetime': '2024-09-19T17:46:28.008272', 'gensim': '4.3.3', 'python': '3.9.19 (main, May  6 2024, 20:12:36) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22631-SP0', 'event': 'created'}


Criando vocabulário

In [9]:
model.build_vocab(train_documents)

2024-09-19 17:46:30,363 : INFO : collecting all words and their counts
2024-09-19 17:46:30,367 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2024-09-19 17:46:30,652 : INFO : PROGRESS: at example #10000, processed 875890 words (3108876 words/s), 17878 word types, 0 tags
2024-09-19 17:46:30,811 : INFO : PROGRESS: at example #20000, processed 1749138 words (5528103 words/s), 24460 word types, 0 tags
2024-09-19 17:46:30,855 : INFO : collected 25996 word types and 22635 unique tags from a corpus of 22635 examples and 1987425 words
2024-09-19 17:46:30,857 : INFO : Creating a fresh vocabulary
2024-09-19 17:46:30,900 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=5 retains 8441 unique words (32.47% of original 25996, drops 17555)', 'datetime': '2024-09-19T17:46:30.900347', 'gensim': '4.3.3', 'python': '3.9.19 (main, May  6 2024, 20:12:36) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22631-SP0', 'event': 'prepare_vocab'}
2024-

In [10]:
model.train(
    train_documents,
    total_examples=model.corpus_count,
    epochs=32
)

2024-09-19 17:46:39,204 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 8 workers on 8442 vocabulary and 4400 features, using sg=0 hs=0 sample=1e-05 negative=20 window=5 shrink_windows=True', 'datetime': '2024-09-19T17:46:39.204411', 'gensim': '4.3.3', 'python': '3.9.19 (main, May  6 2024, 20:12:36) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22631-SP0', 'event': 'train'}
2024-09-19 17:46:40,275 : INFO : EPOCH 0 - PROGRESS: at 8.27% examples, 28679 words/s, in_qsize 15, out_qsize 0
2024-09-19 17:46:41,383 : INFO : EPOCH 0 - PROGRESS: at 20.57% examples, 34005 words/s, in_qsize 15, out_qsize 0
2024-09-19 17:46:42,460 : INFO : EPOCH 0 - PROGRESS: at 32.90% examples, 36094 words/s, in_qsize 15, out_qsize 0
2024-09-19 17:46:43,625 : INFO : EPOCH 0 - PROGRESS: at 44.08% examples, 35936 words/s, in_qsize 16, out_qsize 0
2024-09-19 17:46:44,629 : INFO : EPOCH 0 - PROGRESS: at 55.33% examples, 37014 words/s, in_qsize 16, out_qsize 0
2024-09-19 17:46:45,630 : INFO :

In [11]:
vector = model.infer_vector('gostaria de fazer uma reclamação'.split(' '))
print(vector)

[ 5.8103129e-03  4.0630829e-03  1.0904035e-03  1.6544227e-03
  2.4629582e-03 -9.2914741e-04  2.4596322e-03 -1.4713274e-03
 -7.3296967e-04 -2.7274359e-03 -5.9787091e-04 -2.2587464e-03
  7.0648934e-03  4.1016373e-03 -9.2520221e-04 -3.0426783e-03
  2.1942290e-03  1.4007266e-03 -7.2134738e-03  1.3600122e-03
 -4.5632287e-03 -3.7912668e-03  3.7918966e-03  4.0737623e-03
 -4.6727699e-03 -3.5577577e-03  5.1908744e-03  2.2782332e-03
  1.4538902e-03 -6.5002980e-04  2.6779824e-03  2.2325760e-03
  2.5375285e-03 -2.6460392e-03 -6.6477600e-03 -1.9993363e-03
  3.0528156e-03 -8.8383356e-04 -1.1713987e-03  1.0504816e-03
 -2.2355821e-03 -7.4902078e-04  1.2272883e-03  2.1755509e-03
  8.1326952e-04 -3.1075079e-03 -1.8428810e-03 -9.9664042e-04
  6.2261946e-03  2.1043487e-03 -3.3363504e-03 -1.6753464e-03
  1.4460665e-03 -1.4425744e-03 -2.2568195e-03 -5.2662316e-04
 -1.8517027e-04 -1.3509477e-03 -1.7260720e-03  3.2115659e-05
  7.3214731e-04 -3.2514534e-03 -2.0057256e-03  3.2294043e-03
 -5.5181100e-03  4.59056

Teste de 'sanidade', calculando similaridade de documentos em relação ao conjunto todo

In [12]:
ranks = []
second_ranks = []

for doc_id in range(len(train_documents)):
    inferred_vector = model.infer_vector(train_documents[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

In [15]:
collections.Counter(ranks)[0]

18248

18248 de 22635 reclamações (80,62 %) são similares a elas mesmas, o que é um bom sinal

In [19]:
doc_id = 53

inferred_vector = model.infer_vector(train_documents[doc_id].words)
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))

print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_documents[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_documents[sims[index][0]].words)))

Document (53): «recebi conta tim com vencimento em estou questionando valor primeiro questionamento solicitei portabilidade da tim para oi em novembro de liguei para atendimento tim no dia solicitei que meu numero xxxxx fosse migrado para pre pago como renovacao do meu plano acontece todo dia informei que havia feito portabilidade para oi mesma aconteceria no dia aconteceu no dia as da manha atendente foi categorico dizendo que nao havia necessidade que eu poderia usar ate dia sem que fosse cobrada renovacao do plano desconfiado confirmei novamente foi categorico dizendo que era isso mesmo estou questionando valor de referente ao periodo de gravacao da conversa pode ser consultada atraves do protocolo xxx xxx xxx xx segundo questionamento nao recebi conta tim com vencimento em novembro por mail nao lembrava que estava em debito automatico verifiquei cobranca atraves do dda fiz pagamento em comprovante em anexo no dia aconteceu debito automatico pagando conta com vencimento em dezembro 

In [22]:
semantic_query = 'estou com problemas com meu numero da claro'

inferred_vector = model.infer_vector(semantic_query.split(' '))
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))

print(u'%s %s: «%s»\n' % ('MOST', sims[index], ' '.join(train_documents[sims[index][0]].words)))

MOST (12980, -0.6923020482063293): «relato que desde marco de realizei portabilidade da linha principal as dependentes para outra operadora apos ter sido surpreendido com inumeras cobrancas indevidas alteracoes contratuais de forma unilateral pela reclamada sem minha anuencia sendo assim procurei agencia de protecao defesa do consumidor de juiz de fora conforme os numeros de processos xxxxx xxxxx foi solucionado conforme termo seguir teresina de marco de procon municipal de juiz de fora av presidente itamar franco terreo cep xxxxx juiz de fora mg fa xxx xxx xxx xx cpf xxx xxx xxx xx consumidor jose fernandes de oliveira linha reclamada pedido prezados senhores em resposta ao questionamento do sr jose fernandes de oliveira encaminhada por esse orgao de defesa do consumidor registrado nesta empresa sob protocolo de nº xxx xxx xxx xx informamos que cliente possuia linha xxx xxx xxx xx vinculado ao plano oi mais gb cancelado em informamos que linha fixa velox encontram se devidamente cance

Testando o modelo com o conjunto de teste

In [26]:
# Escolhendo aleatoriamente um documento do conjunto de teste
doc_id = random.randint(0, len(test_documents) - 1)
inferred_vector = model.infer_vector(test_documents[doc_id])
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))

# Comparando este documento com o conjunto de treinamento
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_documents[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_documents[sims[index][0]].words)))

Test Document (4748): «cobranca de fevereiro de de no mes de marco de minha fatura sempre foi de reais ja foi debitado pois debito automatico»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec<dm/c,d400,n20,w5,mc5,s1e-05,t8>:

MOST (3425, 0.9134390950202942): «de acordo com reclamacao numero protocolo xxx xxx xxx xx era pra minha fatura vir reais mas veio no valor normal ja foi debitado da minha conta»

MEDIAN (13560, 0.20986169576644897): «ainda me mandam faturas de um numero que ja esta no pre pago estao me ligando cobrando faturas peco providencias pois isso esta me gerando transtorno estao ameacando de mandar meu nome para spc»

LEAST (14207, -0.9050188660621643): «um ano atras migrei um plano para pre pago ontem fui uma loja infezlimente ainda insiste em me mandar faturas em valores altos negativaram meu nome peco por gentileza as retiradas dessas faturas mais rapido possivel pois preciso do meu nome isso esta me impedindo de abri um financiamento peco que liguem em xxx xxx xxx xx»



Salvando modelo Doc2Vec

In [27]:
model.save('../model/doc2vec_telecom_pandemic_claims')

2024-09-19 19:02:23,668 : INFO : Doc2Vec lifecycle event {'fname_or_handle': '../model/doc2vec_telecom_pandemic_claims', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2024-09-19T19:02:23.667075', 'gensim': '4.3.3', 'python': '3.9.19 (main, May  6 2024, 20:12:36) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22631-SP0', 'event': 'saving'}
2024-09-19 19:02:23,672 : INFO : storing np array 'syn1neg' to ../model/doc2vec_telecom_pandemic_claims.syn1neg.npy
2024-09-19 19:02:23,999 : INFO : not storing attribute cum_table
2024-09-19 19:02:24,126 : INFO : saved ../model/doc2vec_telecom_pandemic_claims
